In [2]:
from dnn_structure import *

In [3]:
import numpy as np
import sys
sys.path.append("../src")
from Load_data.xml_utils import parse_cvat_xml_all_labels, label_Y_binary
from Load_data.data_loader import load_and_resize_images, build_label_array


In [4]:
TRAIN_XML = "../EIDSeg_Dataset/data/train/train.xml"
TEST_XML = "../EIDSeg_Dataset/data/test/test.xml"
TRAIN_IMAGES = "../EIDSeg_Dataset/data/train/images/default"
TEST_IMAGES = "../EIDSeg_Dataset/data/test/images/default"

IMAGE_SIZE = (64, 64)
NUM_ITER = 2000
LR = 0.001

## Load and parse XML labels

In [5]:
labels_train_raw = parse_cvat_xml_all_labels(TRAIN_XML)
labels_test_raw = parse_cvat_xml_all_labels(TEST_XML)

Y_train_map = label_Y_binary(labels_train_raw)
Y_test_map = label_Y_binary(labels_test_raw)

## Load and preprocess images

In [6]:
X_train_org, ordered_train = load_and_resize_images(TRAIN_IMAGES, size=IMAGE_SIZE)
X_test_org, ordered_test = load_and_resize_images(TEST_IMAGES, size=IMAGE_SIZE)

Y_train_org = build_label_array(ordered_train, Y_train_map)
Y_test_org = build_label_array(ordered_test, Y_test_map)

Final X shape: (2612, 64, 64, 3)
Final X shape: (327, 64, 64, 3)


## Flatten Inputs

In [7]:
train_x = X_train_org.reshape(X_train_org.shape[0], -1).T
test_x = X_test_org.reshape(X_test_org.shape[0], -1).T

print(train_x.shape, Y_train_org.shape)
print(test_x.shape, Y_test_org.shape)

(12288, 2612) (1, 2612)
(12288, 327) (1, 327)


## L-layer Neural Network


In [8]:
layers_dims = [12288, 20, 7, 5, 1] 

In [9]:

def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- input data, of shape (n_x, number of examples)
    Y -- true "label" vector (containing 1 if cat, 0 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    # Parameters initialization.

    parameters = initialize_parameters_deep(layers_dims)
        
    # Loop (gradient descent)
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        AL, caches = L_model_forward(X, parameters)
                
        # Compute cost.
        cost = compute_cost(AL,Y)
            
        # Backward propagation.
        grads = L_model_backward(AL, Y, caches)
        
        # Update parameters.

        parameters = update_parameters(parameters, grads, learning_rate)
                        
        # Print the cost every 100 iterations and for the last iteration
        if print_cost and (i % 100 == 0 or i == num_iterations - 1):
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0:
            costs.append(cost)
    
    return parameters, costs

In [ ]:
parameters, costs = L_layer_model(train_x, Y_train_org, layers_dims, num_iterations = 2500, print_cost = True)

Cost after iteration 0: 0.636104091418676
Cost after iteration 100: 0.5821166290497682
Cost after iteration 200: 0.5680560386139355
Cost after iteration 300: 0.5598642204551367
Cost after iteration 400: 0.5527998727197412
Cost after iteration 500: 0.5445039381704909
Cost after iteration 600: 0.5337375044639118
Cost after iteration 700: 0.5416522764104826
Cost after iteration 800: 0.5232626369433263
Cost after iteration 900: 0.5145906320741533
Cost after iteration 1000: 0.5084620409315832
Cost after iteration 1100: 0.5185540249584121
Cost after iteration 1200: 0.525599631712829
Cost after iteration 1300: 0.4793718900127803
Cost after iteration 1400: 0.46298685831653785
Cost after iteration 1500: 0.4716433920133009
Cost after iteration 1600: 0.5207681572443652
Cost after iteration 1700: 0.457336810979845
Cost after iteration 1800: 0.44111478011260247
Cost after iteration 1900: 0.5495052634519821
Cost after iteration 2000: 0.38406075316504124
Cost after iteration 2100: 0.34273613936422137

In [ ]:
pred_train = predict(train_x, Y_train_org, parameters)

In [ ]:
pred_test = predict(test_x, Y_test_org, parameters)